In [1]:
import cython
import numpy as np

In [2]:
%load_ext cython

In [28]:
%%cython --annotate --force
# distutils: language = c++

from libcpp cimport bool as cpp_bool_t

from scipy.constants import G as G_
from TidalPy.radial_solver.nondimensional import non_dimensionalize_physicals, re_dimensionalize_radial_func
from TidalPy.radial_solver.numerical.collapse import collapse_solutions
from TidalPy.radial_solver.numerical.initial import find_initial_guess
from TidalPy.radial_solver.numerical.interfaces import find_interface_func

cdef double G
G = G_

def radial_solver_x(double[:] radius_array, double complex[:] complex_shear_array, double[:] bulk_array,
                    double[:] density_array, double[:] gravity_array, double frequency, double planet_bulk_density,
                    tuple is_solid_by_layer, tuple is_static_by_layer, tuple is_incompressible_by_layer,
                    tuple indices_by_layer,
                    unsigned int degree_l = 2, 
                    double complex[:] surface_boundary_conditions = None,
                    cpp_bool_t solve_load_numbers = False,
                    cpp_bool_t use_kamata = False,
                    int integration_method = 1,
                    double integration_rtol = 1.0e-8,
                    double integration_atol = 1.0e-9, 
                    cpp_bool_t verbose = False,
                    cpp_bool_t nondimensionalize = True,
                    ):
    # General indexing
    cdef Py_ssize_t i
    
    # Pull out key information
    cdef double radius_planet
    cdef Py_ssize_t num_layers
    radius_planet = radius_array[-1]
    num_layers    = len(is_solid_by_layer)
    
    # Non-dimensionalize inputs
    cdef double G_to_use
    
    if nondimensionalize:
        radius_array, gravity_array, density_array, complex_shear_array, bulk_array, frequency, G_to_use = \
            non_dimensionalize_physicals(
                    radius_array, gravity_array, density_array, complex_shear_array, bulk_array, frequency,
                    mean_radius=radius_planet, bulk_density=planet_bulk_density
                    )
    else:
        G_to_use = G
    
    # Find boundary condition at the top of the planet -- this is dependent on the forcing type.
    #     Tides (default here) follow the (y2, y4, y6) = (0, 0, (2l+1)/R) rule
    cdef np.ndarray[np.complex128_t, ndim=1] bc_array
    cdef double complex[:] bc_array_view
    bc_array = np.empty(3, dtype=np.complex128)
    bc_array_view = bc_array
    
    if surface_boundary_condition is None:
        # Assume tides
        for i in range(3):
            if i == 2:
                if nondimensionalize:
                    bc_array_view[i] == (2. * order_l + 1.) / 1.
                else:
                    bc_array_view[i] == (2. * order_l + 1.) / radius_planet
    else:
        # Use user input
        # TODO: Exception if user input is wrong size.
        for i in range(3):
            bc_array_view[i] = surface_boundary_condition[i]
            
        
        if nondimensionalize:
            surface_boundary_condition[2] = (2. * order_l + 1.) / 1.
        else:
            surface_boundary_condition[2] = (2. * order_l + 1.) / radius[-1]
        
    

Content of stdout:
_cython_magic_31ba67716dcc2001fae5b1212bd61b48e04574b2.cpp
C:\Users\joepr\.ipython\cython\_cython_magic_31ba67716dcc2001fae5b1212bd61b48e04574b2.cpp(23225): warning C4551: function call missing argument list
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_31ba67716dcc2001fae5b1212bd61b48e04574b2.cp310-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_31ba67716dcc2001fae5b1212bd61b48e04574b2.cp310-win_amd64.exp
Generating code
Finished generating code

In [19]:
%%cython --annotate --force
# distutils: language = c++
from libcpp cimport bool as cpp_bool_t
cimport numpy as np


def f(np.uint8_t[:] x):
    cdef int len_x, i
    len_x = len(x)
    
    for i in range(len_x):
        if x[i]:
            print('Yes!')
        else:
            print('No :(')

Content of stdout:
_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cpp
C:\Users\joepr\anaconda3\envs\cytest310\lib\site-packages\numpy\core\include\numpy\npy_1_7_deprecated_api.h(14) : Warning Msg: Using deprecated NumPy API, disable it with #define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION
C:\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cpp(18784): warning C4244: '=': conversion from 'Py_ssize_t' to 'int', possible loss of data
C:\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cpp(24004): warning C4551: function call missing argument list
   Creating library C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cp310-win_amd64.lib and object C:\Users\joepr\.ipython\cython\Users\joepr\.ipython\cython\_cython_magic_043318e08c951a8d23cbf784613d282d2dd82ff9.cp310-win_amd64.exp
Generating code
Finished generating code

In [23]:
x = np.linspace(0., 3., 4)
y = x <= 2.
print(x, y)
f(y)

[0.000e+00 1.000e+00 2.000e+00 3.000e+00] [ True  True  True False]
Yes!
Yes!
Yes!
No :(
